Demo
1. get comments by string matching (i.e. search term hits)
2. filter comments to find entity matching
3. print sentiments

# pushshift gets reddit comments

In [1]:
import requests

In [2]:
entity_name = "uber"

In [3]:
r = requests.get('https://api.pushshift.io/reddit/search/comment/?q={qterm}'.format(
    qterm=entity_name
))

comments = r.json()['data']

# huggingface provides ner, sentiment

https://huggingface.co/transformers/usage.html#named-entity-recognition
https://github.com/huggingface/transformers/blob/master/src/transformers/pipelines.py#L1604

In [4]:
from transformers import pipeline

def wrap_assert(f):
    '''sometimes i dont want to deal with exceptions in f'''
    def newf(*args,**kwargs):
        try:
            return f(*args,**kwargs)
        except Exception:
            return None
    return newf

get_ner = wrap_assert(pipeline("ner"))
get_sentiment = wrap_assert(pipeline("sentiment-analysis"))

In [5]:
from transformers.pipelines import SUPPORTED_TASKS
from transformers import AutoTokenizer

def get_ner_tokens(s):
    framework='pt'
    targeted_task = SUPPORTED_TASKS['ner']

    # https://github.com/huggingface/transformers/blob/master/src/transformers/pipelines.py#L1681
    task_class, model_class = targeted_task["impl"], targeted_task[framework]
    model = targeted_task["default"]["model"][framework]

    tokenizer = model
    # Instantiate tokenizer if needed
    if isinstance(tokenizer, (str, tuple)):
        if isinstance(tokenizer, tuple):
            # For tuple we have (tokenizer name, {kwargs})
            tokenizer = AutoTokenizer.from_pretrained(tokenizer[0], **tokenizer[1])
        else:
            tokenizer = AutoTokenizer.from_pretrained(tokenizer)
    
    # encode -> tokens
    return [
        s for s in tokenizer.convert_ids_to_tokens(tokenizer.encode(s)) 
        if not s.startswith('[') and not s.endswith(']')
    ]

In [6]:
entity_name_tokenized = get_ner_tokens(entity_name)

/Users/tianlechen/GitHub/astroturf/.venv/lib/python3.7/site-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [7]:
entity_name_tokenized

['u', '##ber']

In [8]:
def is_hit(ner, entity_name_tokenized):
    ner_wordlist = [d['word'].lower() for d in ner]
    return all(token in ner_wordlist for token in entity_name_tokenized)

# run

In [9]:
linesep = '-'*100
for comment in comments:
    body = comment['body']
    ner = get_ner(body)
    sentiment = get_sentiment(body)
    if is_hit(ner, entity_name_tokenized):
        print (
            '\n'+linesep+\
            '\ncomment:\n{}'.format(body)+\
            '\nner:\n{}'.format(ner)+\
            '\nsentiment:\n{}'.format(sentiment)+\
            '\n'+linesep
        )


----------------------------------------------------------------------------------------------------
comment:
&gt; don’t use Uber eats

Uber's taxi business has took a hit
ner:
[{'word': 'U', 'score': 0.9954677820205688, 'entity': 'I-ORG', 'index': 9}, {'word': '##ber', 'score': 0.997706413269043, 'entity': 'I-ORG', 'index': 10}, {'word': 'U', 'score': 0.9927937984466553, 'entity': 'I-ORG', 'index': 12}, {'word': '##ber', 'score': 0.9976713061332703, 'entity': 'I-ORG', 'index': 13}]
sentiment:
[{'label': 'NEGATIVE', 'score': 0.9728900194168091}]
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
comment:
Yeah but still, people sell him as an Uber progressive pink tide type and he's a third rate caudillo with regressive ideas on lterally every topic
ner:
[{'word': 'U', 'score': 0.8692496418952942, 'entity': 'I-ORG', 'index': 10}, {'word


----------------------------------------------------------------------------------------------------
comment:
&gt; Unlike say a restaurant where you just always have to tip no matter what

In a restaurant, the server is paid so little that tip is almost mandatory. Restaurants should really up their prices and do away with tips. But servers love the tip culture because they make a lot more money/hour than many other jobs that require greater qualifications.

Hairdresser, cab driver, installer, movers, etc...why is there an expectation to tip for just doing their normal job? 

Totally agree on Uber. They still try to keep it that way - tip if you want to, no obligation. But it's a lot more in your face than it used to be.
ner:
[{'word': 'U', 'score': 0.9835100769996643, 'entity': 'I-ORG', 'index': 107}, {'word': '##ber', 'score': 0.9902449250221252, 'entity': 'I-ORG', 'index': 108}]
sentiment:
[{'label': 'POSITIVE', 'score': 0.9733946323394775}]
-----------------------------------------